In [21]:
!pip install torchinfo
!/Users/adityag/Desktop/ffmpegFolder/ffmpeg 

ffmpeg version 6.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

In [22]:
###CNN Training
##s3dg module
!pip install torch
import torch.nn as nn
import torch

class BasicConv3d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv3d, self).__init__()
        self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding,
                              bias=False)  # verify bias false

        # verify defalt value in sonnet
        self.bn = nn.BatchNorm3d(out_planes, eps=1e-3, momentum=0.001, affine=True)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        #print("myshapeafterBasicConv3d",x.shape);
        return x

class STConv3d(nn.Module):
    def __init__(self,in_planes,out_planes,kernel_size,stride,padding=0):
        super(STConv3d, self).__init__()
        self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=(1,kernel_size,kernel_size),stride=(1,stride,stride),padding=(0,padding,padding))
        self.conv2 = nn.Conv3d(out_planes,out_planes,kernel_size=(kernel_size,1,1),stride=(1,1,1),padding=(padding,0,0))

        self.bn=nn.BatchNorm3d(out_planes, eps=1e-3, momentum=0.001, affine=True)
        self.relu = nn.ReLU(inplace=True)

        self.bn2=nn.BatchNorm3d(out_planes, eps=1e-3, momentum=0.001, affine=True)
        self.relu2=nn.ReLU(inplace=True)

        nn.init.normal(self.conv2.weight,mean=0,std=0.01)
        nn.init.constant(self.conv2.bias,0)

    def forward(self,x):
        x=self.conv(x)
        x=self.bn(x)
        x=self.relu(x)
        x=self.conv2(x)
        x=self.bn2(x)
        x=self.relu2(x)
        #print("myshapeafterSTConv3d",x.shape);
        return x

# Note the operations here for S3D-G:
# If we set two convs: 1xkxk + kx1x1, it's as follows: (p=(k-1)/2)
# BasicConv3d(input,output,kernel_size=(1,k,k),stride=1,padding=(0,p,p))
# Then BasicConv3d(output,output,kernel_size=(k,1,1),stride=1,padding=(p,0,0))

class Mixed_3b(nn.Module):
    def __init__(self):
        super(Mixed_3b, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(192, 64, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(192, 96, kernel_size=1, stride=1),
            STConv3d(96, 128, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(192, 16, kernel_size=1, stride=1),
            STConv3d(16, 32, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(192, 32, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)

        return out


class Mixed_3c(nn.Module):
    def __init__(self):
        super(Mixed_3c, self).__init__()
        self.branch0 = nn.Sequential(
            BasicConv3d(256, 128, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(256, 128, kernel_size=1, stride=1),
            STConv3d(128, 192, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(256, 32, kernel_size=1, stride=1),
            STConv3d(32, 96, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(256, 64, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Mixed_4b(nn.Module):
    def __init__(self):
        super(Mixed_4b, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(480, 192, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(480, 96, kernel_size=1, stride=1),
            STConv3d(96, 208, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(480, 16, kernel_size=1, stride=1),
            STConv3d(16, 48, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(480, 64, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Mixed_4c(nn.Module):
    def __init__(self):
        super(Mixed_4c, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(512, 160, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(512, 112, kernel_size=1, stride=1),
            STConv3d(112, 224, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(512, 24, kernel_size=1, stride=1),
            STConv3d(24, 64, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(512, 64, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Mixed_4d(nn.Module):
    def __init__(self):
        super(Mixed_4d, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(512, 128, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(512, 128, kernel_size=1, stride=1),
            STConv3d(128, 256, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(512, 24, kernel_size=1, stride=1),
            STConv3d(24, 64, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(512, 64, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Mixed_4e(nn.Module):
    def __init__(self):
        super(Mixed_4e, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(512, 112, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(512, 144, kernel_size=1, stride=1),
            STConv3d(144, 288, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(512, 32, kernel_size=1, stride=1),
            STConv3d(32, 64, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(512, 64, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Mixed_4f(nn.Module):
    def __init__(self):
        super(Mixed_4f, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(528, 256, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(528, 160, kernel_size=1, stride=1),
            STConv3d(160, 320, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(528, 32, kernel_size=1, stride=1),
            STConv3d(32, 128, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(528, 128, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Mixed_5b(nn.Module):
    def __init__(self):
        super(Mixed_5b, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(832, 256, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(832, 160, kernel_size=1, stride=1),
            STConv3d(160, 320, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(832, 32, kernel_size=1, stride=1),
            STConv3d(32, 128, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(832, 128, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Mixed_5c(nn.Module):
    def __init__(self):
        super(Mixed_5c, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv3d(832, 384, kernel_size=1, stride=1),
        )
        self.branch1 = nn.Sequential(
            BasicConv3d(832, 192, kernel_size=1, stride=1),
            STConv3d(192, 384, kernel_size=3, stride=1, padding=1),
        )
        self.branch2 = nn.Sequential(
            BasicConv3d(832, 48, kernel_size=1, stride=1),
            STConv3d(48, 128, kernel_size=3, stride=1, padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool3d(kernel_size=(3, 3, 3), stride=1, padding=1),
            BasicConv3d(832, 128, kernel_size=1, stride=1),
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out

class S3DG(nn.Module):

    # def __init__(self, num_classes=512, dropout_keep_prob = 1, input_channel = 3, spatial_squeeze=True):
    def __init__(self, num_classes=512, dropout_keep_prob = 1, input_channel = 1):
        super(S3DG, self).__init__()
        self.features = nn.Sequential(
            STConv3d(input_channel, 64, kernel_size=7, stride=2, padding=3), # ({32 -> batch size} , 64, 16, 112, 112)
            nn.MaxPool3d(kernel_size=(1,3,3), stride=(1,2,2), padding=(0,1,1)),  # (64, 16, 56, 56)
            BasicConv3d(64, 64, kernel_size=1, stride=1), # (64, 16, 56, 56)
            STConv3d(64, 192, kernel_size=3, stride=1, padding=1),  # (192, 16, 56, 56)
            nn.MaxPool3d(kernel_size=(1,3,3), stride=(1,2,2), padding=(0,1,1)),  # (192, 16, 28, 28)
            Mixed_3b(), # (256, 16, 28, 28)
            Mixed_3c(), # (480, 16, 28, 28)
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1,2,2), padding=(0, 1, 1)), # (480, 16, 14, 14)
            Mixed_4b(),# (512, 16, 14, 14)
            Mixed_4c(),# (512, 16, 14, 14)
            Mixed_4d(),# (512, 16, 14, 14)
            Mixed_4e(),# (528, 16, 14, 14)
            Mixed_4f(),# (832, 16, 14, 14)
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1,2,2), padding=(0, 1, 1)), # (832, 16, 7, 7)
            Mixed_5b(), # (832, 16, 7, 7)
            Mixed_5c(), # (1024, 16, 7, 7)
            nn.MaxPool3d(kernel_size=(16, 1, 1), stride=1),# (1024, 1, 7, 7)
            nn.AdaptiveAvgPool3d((1, 1, 1)),##adaptiveAvgPool3d#(1024,1,1,1)
            nn.Conv3d(1024, num_classes, kernel_size=1, stride=1, bias=True),# (512, 1, 1, 1)
            nn.Dropout3d(dropout_keep_prob),#(512)
            nn.Flatten(),
            nn.Linear(512,1),#(1)

        )
        # self.spatial_squeeze = spatial_squeeze
        # self.softmax = nn.Softmax()

    def forward(self, x):
        logits = self.features(x)
        #print("outputshape=",logits.shape);
        # if self.spatial_squeeze:
        #     logits = logits.squeeze(3)
        #     logits = logits.squeeze(3)

        # averaged_logits = torch.mean(logits, 2)
        # predictions = self.softmax(averaged_logits)

        # return predictions, averaged_logits

        return logits;


    # def load_state_dict(self,path):
    #     target_weights=torch.load(path)
    #     own_state=self.state_dict()

    #     for name, param in target_weights.items():

    #         if name in own_state:
    #             if isinstance(param,nn.Parameter):
    #                 param=param.data
    #             try:
    #                 if len(param.size())==5 and param.size()[3] in [3,7]:
    #                     own_state[name][:,:,0,:,:]=torch.mean(param,2)
    #                 else:
    #                     own_state[name].copy_(param)
    #             except Exception:
    #                 raise RuntimeError('While copying the parameter named {}.\
    #                                    whose dimensions in the model are {} and \
    #                                    whose dimensions in the checkpoint are {}.\
    #                                    '.format(name,own_state[name].size(),param.size()))
    #         else:
    #            print ('{} meets error in locating parameters'.format(name))
    #     missing=set(own_state.keys())-set(target_weights.keys())

    #     print ('{} keys are not holded in target checkpoints'.format(len(missing)))


# if __name__=='__main__':

#     model=S3DG(num_classes=512)
#     model(input)

#     # Initialize the weights with pretrained I3D net. In detail,
#     # please refer to specific reproduced load_state_dict() function
#     if not os.path.exists('modelweights/RGB_imagenet.pkl'):
#         print ('No weights Found! please download first, or comment 382~384th line')
#     model.load_state_dict('modelweights/RGB_imagenet.pkl')
#     model=model.cuda()
#     data=torch.autograd.Variable(torch.rand(2,3,64,224,224))#.cuda())
#     out=model(data)[0]
#     print (out.size())


In [23]:
"""
    Clears the contents of a folder and moves a specified number of videos from another folder to it.

    Args:
        clear_folder (str): The folder to clear.
        source_folder (str): The folder with videos to move.
        num_videos_to_move (int): The number of videos to move (default is 16).
    """
import os
import shutil
import random

def clear_folder_and_move_videos(clear_folder, source_folder, num_videos_to_move=16):
    if not os.path.exists(clear_folder):
        os.makedirs(clear_folder)

    for item in os.listdir(clear_folder):
        item_path = os.path.join(clear_folder, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

    video_files = [file for file in os.listdir(source_folder) if file.endswith((".mp4", ".avi", ".mkv", ".mov"))]

    if num_videos_to_move > len(video_files):
        print("The number of videos to move exceeds the total number of videos in the source folder.")
        return False;

    selected_videos = random.sample(video_files, num_videos_to_move)
    for video in selected_videos:
        source_path = os.path.join(source_folder, video)
        destination_path = os.path.join(clear_folder, video)
        shutil.move(source_path, destination_path)

    print(f"{num_videos_to_move} videos have been randomly selected and moved to the clear folder.")
    return True;

#clear_folder_and_move_videos("/Users/adityag/Desktop/BTP/trainFolder", "/Users/adityag/Desktop/BTP/dataset/trainClipped35224", 3)


In [24]:
##deleting files in a folder
import os
def empty_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            print(file_path);
            os.remove(file_path)


In [25]:
#increasing fps by interpolating videos
inputCount=0;
import os
def Interpolating_folder(folder_path):
    empty_folder("/Users/adityag/Desktop/BTP/trainFolderInterpolated")
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            print("file=",file);
            file_path = os.path.join(root, file)
            print("file_path=",file_path);
            i=0;
            num1="";
            for i in range(0,100000000,1):
              if(file[i]=='v'):
                break;
              num1+=file[i];
              i=i+1;
            num2=file[i+6];
            num1=num1.split("/")[-1];
            print("num1="+ num1 , "num2=" , num2);
            global inputCount
            inputCount+=1;
            if((inputCount%3)==0):
                varName40="/Users/adityag/Desktop/BTP/trainFolderInterpolated/"+"Interpolated"+num1+"_"+num2+"_"+inputCount+"_40fps"+"Video.mp4";
                !/Users/adityag/Desktop/ffmpegFolder/ffmpeg -loglevel quiet -i "$file_path" -vf "minterpolate='mi_mode=mci:mc_mode=aobmc:vsbmc=1:fps=40'" -c:a copy "$varName40"
            elif((inputCount%3)==1):
                varName50="/Users/adityag/Desktop/BTP/trainFolderInterpolated/"+"Interpolated"+num1+"_"+num2+"_"+inputCount+"_"+"50fps"+"Video.mp4";
                !/Users/adityag/Desktop/ffmpegFolder/ffmpeg -loglevel quiet -i "$file_path" -vf "minterpolate='mi_mode=mci:mc_mode=aobmc:vsbmc=1:fps=50'" -c:a copy "$varName50"
            else:
                varName60="/Users/adityag/Desktop/BTP/trainFolderInterpolated/"+"Interpolated"+num1+"_"+num2+"_"+inputCount+"_60fps"+"Video.mp4";
                !/Users/adityag/Desktop/ffmpegFolder/ffmpeg -loglevel quiet -i "$file_path" -vf "minterpolate='mi_mode=mci:mc_mode=aobmc:vsbmc=1:fps=60'" -c:a copy "$varName60"

# folder_path = '/Users/adityag/Desktop/BTP/trainFolder'
# Interpolating_folder(folder_path)


In [26]:
import os
import shutil

def copy_files_from_folders_to_destination(source_folder1, source_folder2, destination_folder):
    """
    Clears the destination folder and copies files from two source folders to the destination folder.
    
    Args:
        source_folder1 (str): Path to the first source folder.
        source_folder2 (str): Path to the second source folder.
        destination_folder (str): Path to the destination folder.
    """
    # Ensure the destination folder exists, create it if not
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Function to clear the destination folder
    def clear_destination_folder(destination_folder):
        for item in os.listdir(destination_folder):
            item_path = os.path.join(destination_folder, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
    
    # Clear the destination folder
    clear_destination_folder(destination_folder)
    
    # Function to copy files from source folders to the destination folder
    def copy_files(source_folder, destination_folder):
        for item in os.listdir(source_folder):
            item_path = os.path.join(source_folder, item)
            destination_path = os.path.join(destination_folder, item)
            if os.path.isfile(item_path):
                shutil.copy(item_path, destination_path)
            elif os.path.isdir(item_path):
                shutil.copytree(item_path, destination_path)
    
    # Copy files from the first source folder
    copy_files(source_folder1, destination_folder)
    
    # Copy files from the second source folder
    copy_files(source_folder2, destination_folder)
    
    print("Files have been copied to the destination folder.")

# source_folder1 = "/Users/adityag/Desktop/BTP/trainFolder"
# source_folder2 = "/Users/adityag/Desktop/BTP/trainFolderInterpolated"
# destination_folder = "/Users/adityag/Desktop/BTP/trainFolderFinal"
# copy_files_from_folders_to_destination(source_folder1, source_folder2, destination_folder)


In [38]:
##preprocesssing
##extracting first 17 grayscale frames
##training resizing and centre cropping
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A

def center_crop_frames(frames, crop_size=224):
    cropped_frames = []
    transform = A.CenterCrop(height=crop_size, width=crop_size)
    for frame in frames:
        transformed = transform(image=frame.astype(np.uint8))
        cropped_frame = transformed["image"].astype(np.float32)
        cropped_frames.append(cropped_frame)
    return np.array(cropped_frames)


def resize_frames(frames):
    resized_frames = []
    r = np.random.uniform(1.0, 1.1)##not (0.9,1,1) bcz some frames becmae small than 224*224
    for frame in frames:
        transform = A.Compose([
            A.Resize(int(r * len(frame)), int(r * len(frame[0]))),
        ])
        transformed = transform(image=frame.astype(np.uint8))
        resized_frame = transformed["image"].astype(np.float32)
        resized_frames.append(resized_frame)
    return np.array(resized_frames ,dtype=object)


def extract_frames_from_video(video_path, num_frames=17):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return None
    frames = []
    for i in range(num_frames):
        ret, frame = cap.read()
        if not ret:
            print(f"Error reading frame {i} from video: {video_path}")
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray_frame)
    cap.release()
    return frames


def extract_frames_from_folder(folder_path, num_frames=17):
    all_frames = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in ['.mp4', '.avi', '.mkv']):
            video_frames = extract_frames_from_video(file_path, num_frames)
            if video_frames is not None:
                all_frames.extend(video_frames)
    frames_array = np.array(all_frames ,dtype=object)
    return frames_array




# folder_to_process = '/Users/adityag/Desktop/BTP/trainFolderFinal'
# frames_array = extract_frames_from_folder(folder_to_process, num_frames=17)
# resized_frames = resize_frames(frames_array)
# cropped_frames = center_crop_frames(resized_frames, crop_size=224)


# print("Frames array shape:", frames_array.shape)
# print("Resized Frames array shape:", resized_frames.shape)
# print("Cropped Frames array shape:", cropped_frames.shape)
# print(len(resized_frames));
# print(len(resized_frames[0]));
# print(len(resized_frames[0][0]));
# print(len(resized_frames[1]));
# print(len(resized_frames[1][0]));
# print(np.shape(resized_frames))
# print(resized_frames[21]);
# print("first frame:=",frames_array[21])

In [28]:
###S_FRA
###S_RES
###S_OPT

import cv2 as cv
import numpy as np


def exclude_indices_3d(original_array, indices_to_exclude):
    mask = np.ones(original_array.shape[0], dtype=bool)
    mask[indices_to_exclude] = False
    new_array = original_array[mask]
    return new_array


def S_fra_creator(cropped_frames):
    indices_to_exclude_s_fra = []
    for i in range(17,cropped_frames.shape[0]+5,17):
      indices_to_exclude_s_fra.append(i-1);
    S_fra = exclude_indices_3d(cropped_frames, indices_to_exclude_s_fra)
    return S_fra;



def S_res_creator(cropped_frames):
    S_res=cropped_frames;
    indices_to_exclude_s_res = []
    for i in range(0,cropped_frames.shape[0],1):
      if((i%17)==0):
        indices_to_exclude_s_res.append(i);
        continue;
      else:
        S_res[i]=cropped_frames[i]-cropped_frames[i-1];
    S_res = exclude_indices_3d(cropped_frames, indices_to_exclude_s_res)
    return S_res;

def S_opt_creator(cropped_frames):
    frame_count=len(cropped_frames)
    first_frame = cropped_frames[0]
    first_frame_rgb= cv2.cvtColor(first_frame, cv2.COLOR_GRAY2RGB)
    mask = np.zeros_like(first_frame_rgb)
    index=1;
    S_opt=[];
    for index in range(1 ,frame_count ,1):
        if((index%17)==0):
            continue;
        frame = cropped_frames[index];
        prev_gray=cropped_frames[i-1];
        gray = frame
        mask[..., 1] = 255
        flow = cv.calcOpticalFlowFarneback(prev_gray, gray,
                                        None,
                                        0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
        mask[..., 0] = angle * 180 / np.pi / 2
        mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
        rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
        gray_image = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY);
        S_opt.append(gray_image);
    S_opt=np.array(S_opt)
    return S_opt;
    

    

# S_fra = S_fra_creator(cropped_frames)
# S_res=S_res_creator(cropped_frames)
# S_opt=S_opt_creator(cropped_frames)
# print("S_fra_shape=",S_fra.shape)
# print("S_res_shape=",S_res.shape)
# print("S_opt_shape=",S_opt.shape)

In [36]:
ideal_target=[];
for i in range(0,16,1):
    ideal_target.append(0);
for i in range(0,16,1):
    ideal_target.append(1);
ideal_target=np.array(ideal_target);



In [34]:
import os
import shutil
import random

def copyShuffle(desitnationFolderEpoch , SourceFolderEpoch):
    empty_folder(desitnationFolderEpochh);
    files = os.listdir(SourceFolderEpoch)
    random.shuffle(files)
    for file in files:
        source_path = os.path.join(SourceFolderEpoch, file)
        destination_path = os.path.join(desitnationFolderEpoch, file)
        if os.path.isfile(source_path):
            shutil.copy(source_path, destination_path)
    print("Files have been copied to the destination folder in shuffled order.")


##copyShuffle()
    

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

model = S3DG()
# Define Binary Cross Entropy loss and Adam optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Check if GPU is available and move the model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
branch_model = model.to(device)
# Learning rate scheduler: ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Example training loop
epochs = 1  # Set your desired number of epochs
for epoch in range(epochs):
    # Example: Get training data (inputs, targets) for each epoch
    ###########INPUT HERE
    
    destFolder="/Users/adityag/Desktop/BTP/trainFolder";
    SourceFolder="/Users/adityag/Desktop/BTP/dataset/trainClipped35224";
    while(clear_folder_and_move_videos(destFolder ,SourceFolder,16)):
        Interpolating_folder('/Users/adityag/Desktop/BTP/trainFolder')
        copy_files_from_folders_to_destination("/Users/adityag/Desktop/BTP/trainFolder",
                                           "/Users/adityag/Desktop/BTP/trainFolderInterpolated",
                                           "/Users/adityag/Desktop/BTP/trainFolderFinal")
        frames_array = extract_frames_from_folder('/Users/adityag/Desktop/BTP/trainFolderFinal',
                                              num_frames=17)
        resized_frames = resize_frames(frames_array)
        cropped_frames = center_crop_frames(resized_frames, crop_size=224)
        S_fra = S_fra_creator(cropped_frames)
        S_res=S_res_creator(cropped_frames)
        S_opt=S_opt_creator(cropped_frames)
        reshaped_array = S_fra.reshape(32, 1, 16, *S_fra.shape[1:])
        inputs = torch.Tensor(reshaped_array)
        ###TARGET
        targets=ideal_target.reshape(32,1);
        targets=torch.Tensor(targets);
    
        ###inputs = torch.randn(32, 1, 16, 224, 224);
        ###inputs=inputs.reshape(32, 1, 16, 224, 224);
        ###targets = torch.randint(0, 2, (32,), dtype=torch.float)b 

        # Move data to GPU if available
        inputs, targets = inputs.to(device), targets.to(device)
        # Forward pass
        logits = model(inputs)
        loss = criterion(logits, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Learning rate scheduler step
    scheduler.step(loss)

    # Print loss for monitoring
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Learning Rate: {optimizer.param_groups[0]["lr"]}')
    copyShuffle("/Users/adityag/Desktop/BTP/dataset/trainClipped35224",
                "/Users/adityag/Desktop/BTP/dataset/trainClippedFinal35224")

# After training, you can use the branch_model for predictions


/var/folders/t7/7hwtsy1n0g1715k_68jkd14r0000gw/T/ipykernel_16352/1775924089.py:36: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.conv2.weight,mean=0,std=0.01)
/var/folders/t7/7hwtsy1n0g1715k_68jkd14r0000gw/T/ipykernel_16352/1775924089.py:37: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.conv2.bias,0)


16 videos have been randomly selected and moved to the clear folder.
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated5401_1_40fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated897_1_40fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated8843_1_50fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated4435_3_40fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated2619_2_40fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated5352_1_40fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated3844_1_60fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated9734_1_50fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated7154_2_40fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated3111_1_50fpsVideo.mp4
/Users/adityag/Desktop/BTP/trainFolderInterpolated/Interpolated8809_1_50fpsVideo.m

file= 8987video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/8987video_1_3.mp4
num1=8987 num2= 1
file= 9104video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9104video_1_1.mp4
num1=9104 num2= 1
file= 9587video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9587video_1_2.mp4
num1=9587 num2= 1
file= 5187video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/5187video_1_3.mp4
num1=5187 num2= 1
file= 8555video_1_4.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/8555video_1_4.mp4
num1=8555 num2= 1
file= 7218video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/7218video_1_3.mp4
num1=7218 num2= 1
file= 4143video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/4143video_2_1.mp4
num1=4143 num2= 2
file= 7184video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/7184video_2_1.mp4
num1=7184 num2= 2
file= 10964video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/10964video_1_1.mp4
num1=10964 num2= 1
file= 1

file= 4324video_1_4.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/4324video_1_4.mp4
num1=4324 num2= 1
file= 10674video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/10674video_1_2.mp4
num1=10674 num2= 1
file= 5597video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/5597video_1_1.mp4
num1=5597 num2= 1
file= 3777video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3777video_1_1.mp4
num1=3777 num2= 1
file= 4970video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/4970video_1_2.mp4
num1=4970 num2= 1
file= 141video_1_5.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/141video_1_5.mp4
num1=141 num2= 1
file= 1270video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/1270video_2_1.mp4
num1=1270 num2= 2
file= 6384video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/6384video_1_1.mp4
num1=6384 num2= 1
file= 9914video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9914video_1_2.mp4
num1=9914 num2= 1
file= 8612

file= 1526video_3_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/1526video_3_1.mp4
num1=1526 num2= 3
file= 8926video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/8926video_1_2.mp4
num1=8926 num2= 1
file= 2397video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2397video_1_1.mp4
num1=2397 num2= 1
file= 5185video_2_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/5185video_2_2.mp4
num1=5185 num2= 2
file= 8717video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/8717video_1_1.mp4
num1=8717 num2= 1
file= 9711video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9711video_1_1.mp4
num1=9711 num2= 1
file= 8399video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/8399video_1_1.mp4
num1=8399 num2= 1
file= 4764video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/4764video_1_1.mp4
num1=4764 num2= 1
file= 7929video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/7929video_1_1.mp4
num1=7929 num2= 1
file= 5692

...

file= 6895video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/6895video_1_1.mp4
num1=6895 num2= 1
file= 2953video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2953video_1_2.mp4
num1=2953 num2= 1
file= 2823video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2823video_1_2.mp4
num1=2823 num2= 1
file= 5483video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/5483video_1_1.mp4
num1=5483 num2= 1
file= 5114video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/5114video_2_1.mp4
num1=5114 num2= 2
file= 5918video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/5918video_1_1.mp4
num1=5918 num2= 1
file= 1951video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/1951video_1_2.mp4
num1=1951 num2= 1
file= 3337video_2_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3337video_2_2.mp4
num1=3337 num2= 2
file= 6043video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/6043video_1_1.mp4
num1=6043 num2= 1
file= 3907

file= 1238video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/1238video_2_1.mp4
num1=1238 num2= 2
file= 10239video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/10239video_1_1.mp4
num1=10239 num2= 1
file= 7757video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/7757video_1_1.mp4
num1=7757 num2= 1
file= 9451video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9451video_1_1.mp4
num1=9451 num2= 1
file= 8893video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/8893video_1_1.mp4
num1=8893 num2= 1
file= 9617video_1_4.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9617video_1_4.mp4
num1=9617 num2= 1
file= 6661video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/6661video_1_2.mp4
num1=6661 num2= 1
file= 3872video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3872video_1_2.mp4
num1=3872 num2= 1
file= 9646video_1_4.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9646video_1_4.mp4
num1=9646 num2= 1
file= 2

file= 8549video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/8549video_1_3.mp4
num1=8549 num2= 1
file= 2089video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2089video_1_2.mp4
num1=2089 num2= 1
file= 3837video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3837video_2_1.mp4
num1=3837 num2= 2
file= 3946video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3946video_1_3.mp4
num1=3946 num2= 1
file= 1292video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/1292video_1_3.mp4
num1=1292 num2= 1
file= 4691video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/4691video_2_1.mp4
num1=4691 num2= 2
file= 6936video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/6936video_1_1.mp4
num1=6936 num2= 1
file= 6947video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/6947video_1_3.mp4
num1=6947 num2= 1
file= 2898video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2898video_2_1.mp4
num1=2898 num2= 2
file= 9500

file= 6505video_1_4.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/6505video_1_4.mp4
num1=6505 num2= 1
file= 9138video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9138video_1_2.mp4
num1=9138 num2= 1
file= 3365video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3365video_1_2.mp4
num1=3365 num2= 1
file= 769video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/769video_1_2.mp4
num1=769 num2= 1
file= 9367video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9367video_1_2.mp4
num1=9367 num2= 1
file= 7081video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/7081video_1_1.mp4
num1=7081 num2= 1
file= 3922video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3922video_1_3.mp4
num1=3922 num2= 1
file= 2323video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2323video_1_2.mp4
num1=2323 num2= 1
file= 3615video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3615video_1_2.mp4
num1=3615 num2= 1
file= 4726vid

file= 1142video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/1142video_2_1.mp4
num1=1142 num2= 2
file= 2013video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2013video_1_2.mp4
num1=2013 num2= 1
file= 3155video_2_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/3155video_2_2.mp4
num1=3155 num2= 2
file= 9573video_1_3.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9573video_1_3.mp4
num1=9573 num2= 1
file= 2296video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/2296video_1_1.mp4
num1=2296 num2= 1
file= 9244video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9244video_2_1.mp4
num1=9244 num2= 2
file= 4983video_2_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/4983video_2_1.mp4
num1=4983 num2= 2
file= 9509video_1_2.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9509video_1_2.mp4
num1=9509 num2= 1
file= 9383video_1_1.mp4
file_path= /Users/adityag/Desktop/BTP/trainFolder/9383video_1_1.mp4
num1=9383 num2= 1
file= 1977

In [10]:
##clear and transfer retuns bool
#after each epoch copy with shuffling
##dropout prob ??